### Part of code related to temperature scaling calibration and human expert simulation is taken from paper "Combining Human Predictions with Model Probabilities via Confusion Matrices and Calibration" code git at link: https://github.com/GavinKerrigan/conf_matrix_and_calibration

In [ ]:
!pip install attrdict deepdish pyro-ppl uncertainty-calibration
!git clone https://github.com/GavinKerrigan/conf_matrix_and_calibration.git
!mv /content/conf_matrix_and_calibration/cifar10h .
!rm -R /content/conf_matrix_and_calibration
!mkdir output
!mv data.csv cifar10h/

In [2]:
from tqdm.auto import tqdm
import torch
from sklearn.model_selection import train_test_split
import csv
import numpy as np
import os

In [3]:
import warnings
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [4]:
import numpy as np
from scipy import optimize
from sklearn.isotonic import IsotonicRegression

In [5]:
from torch import nn, optim
from torch.distributions.log_normal import LogNormal
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
from torch.nn.functional import softmax

In [6]:
# from attrdict import AttrDict
from sklearn.cluster import KMeans 

In [7]:
def load_CIFAR10H(model_name):
    """ Loads the CIFAR-10H predictions (human and model) and true labels.
    """
    if model_name == 'r_low_acc':
        data_path = 'cifar10h/human_model_truth_cifar10h.csv'
        data = np.genfromtxt(data_path, delimiter=',')

        human_counts = data[:, :10]
        model_probs = data[:, 10:20]
        true_labels = data[:, -1]

        true_labels -= 1  # data has labels 1-10 -- shifting so that they are zero-indexed.
    else:
        data_path = f'cifar10h/{model_name}.csv'
        data = np.genfromtxt(data_path, delimiter=',')

        true_labels = data[:, 0]
        human_counts = data[:, 1:11]
        model_probs = data[:, 11:]

    true_labels = true_labels.astype(int)

    return human_counts, model_probs, true_labels

In [8]:
# Function to print accuracy of each class of CIFAR-10. 
def print_classwise_accuracy(y_val, Y_pred, classes = []):
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    for label, prediction in zip(y_val, Y_pred):
        if label == prediction:
            correct_pred[classes[label]] += 1
        total_pred[classes[label]] += 1


    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / (total_pred[classname] + 0.00001)
        print(f'Accuracy for class: {classname:5s} of size {total_pred[classname]} is {accuracy:.1f} %')

In [9]:
human_counts, model_probs, true_labels = load_CIFAR10H('r_low_acc')
y_pred = np.argmax(model_probs, axis=1)
print_classwise_accuracy(true_labels, y_pred)

Accuracy for class: plane of size 1000 is 90.3 %
Accuracy for class: car   of size 1000 is 94.6 %
Accuracy for class: bird  of size 1000 is 85.9 %
Accuracy for class: cat   of size 1000 is 75.0 %
Accuracy for class: deer  of size 1000 is 89.8 %
Accuracy for class: dog   of size 1000 is 84.0 %
Accuracy for class: frog  of size 1000 is 92.0 %
Accuracy for class: horse of size 1000 is 90.9 %
Accuracy for class: ship  of size 1000 is 93.8 %
Accuracy for class: truck of size 1000 is 92.9 %


In [10]:
import random

# Class to simulate expert which has 100% accuracy for k class out of n_classes 
class synth_expert_for_k_class:
    '''
    simple class to describe our synthetic expert on CIFAR-10
    ----
    k: number of classes expert can predict
    n_classes: number of classes (10+1 for CIFAR-10)
    '''
    def __init__(self, k, n_classes):
        self.k = k
        self.n_classes = n_classes

    def predict(self, labels):
        batch_size = labels.shape[0]  # batch_size
        outs = [0] * batch_size
        for i in range(0, batch_size):
            if labels[i] <= self.k:
                outs[i] = labels[i]
            else:
                prediction_rand = random.randint(0, self.n_classes - 1)
                outs[i] = prediction_rand
        return np.array(outs)

In [13]:
k = 5
n_classes = 10 # for CIFAR-10H dataset
expert = synth_expert_for_k_class(k, n_classes)
y_h = expert.predict(true_labels)
print('Expert accuracy: {}'.format(np.mean(y_h == true_labels)))

Expert accuracy: 0.6402


In [14]:
# Function to simulate expert which has overall human expert accuracy to be human_acc accross all classes
def synthetic_expert(y_true, human_acc):
    y_h = []
    size = y_true.shape[0]
    random_values = np.random.rand(size)
    human_acc = 0.5
    for i in range(size):
        if(human_acc < random_values[i]):
            y_h.append(np.random.randint(10))
        else:
            y_h.append(y_true[i])

    y_h = np.array(y_h)
    return y_h

In [15]:
# Function to show the confidence distribution of the neural. Used to show that model machine learning model are not well calibrated
def confidence_distribution(model_probs_te, y_true_te):    
    # Confidence distribution before calibration 
    Y_pred = np.argmax(model_probs_te, axis=1)
    confidence_count = [0]*101
    correct_count = [0]*101
    size = y_true_te.shape[0]
    for i in range(size):
        index = math.ceil(100 * model_probs_te[i][Y_pred[i]])
        confidence_count[index] += 1
        if Y_pred[i] == y_true_te[i]:
            correct_count[index] += 1

    d_correct = np.array([corr/(count+1e-30) for corr , count in zip(correct_count, confidence_count)])
    return d_correct

In [16]:
import math
def classwise_confidence_distribution(model_probs_te, y_true_te):    
    # Confidence distribution before calibration 
    Y_pred = np.argmax(model_probs_te, axis=1)
    confidence_count = [0]*101
    correct_count = [0]*101
    size = y_true_te.shape[0]
    for i in range(size):
        if y_true_te[i] == 1:
            index = math.ceil(100 * model_probs_te[i][Y_pred[i]])
            confidence_count[index] += 1
            if Y_pred[i] == y_true_te[i]:
                correct_count[index] += 1

    d_correct = np.array([corr/(count+1e-30) for corr , count in zip(correct_count, confidence_count)])
    return d_correct

In [17]:
class TSCalibrator():
    """ Maximum likelihood temperature scaling (Guo et al., 2017)
    """

    def __init__(self, temperature=1., n_clusters=14):
        super().__init__()
        self.temperature = []
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=0)

        self.loss_trace = None

    def fitHelper(self, logits, y, cluster):
        """ Fits temperature scaling using hard labels.
        """
        # Pre-processing
        self.n_classes = logits.shape[1]
        _model_logits = torch.from_numpy(logits)
        _y = torch.from_numpy(y)
        _temperature = torch.tensor(1.0, requires_grad=True)

        # Optimization parameters
        nll = nn.CrossEntropyLoss()  # Supervised hard-label loss
        num_steps = 7500
        learning_rate = 0.05
        grad_tol = 1e-3  # Gradient tolerance for early stopping
        min_temp, max_temp = 1e-2, 1e4  # Upper / lower bounds on temperature

        optimizer = optim.Adam([_temperature], lr=learning_rate)

        loss_trace = []  # Track loss over iterations
        step = 0
        converged = False
        while not converged:

            optimizer.zero_grad()
            loss = nll(_model_logits / _temperature, _y)
            loss.backward()
            optimizer.step()
            loss_trace.append(loss.item())

            with torch.no_grad():
                _temperature.clamp_(min=min_temp, max=max_temp)

            step += 1
            if step > num_steps:
                warnings.warn('Maximum number of steps reached -- may not have converged (TS)')
            converged = (step > num_steps) or (np.abs(_temperature.grad) < grad_tol)

        self.loss_trace = loss_trace
        # self.temperature = _temperature.item()
        return _temperature.item()

    def fit(self, model_logits, y_true):
        self.kmeans.fit(model_logits)

        _model_logits = []
        _y_true = []
        for _ in range(self.n_clusters):
            _model_logits.append([])
            _y_true.append([])
            self.temperature.append(1.0)

        for i in range(self.kmeans.labels_.shape[0]):
            cluster_center = self.kmeans.labels_[i]
            _model_logits[cluster_center].append(model_logits[i])
            _y_true[cluster_center].append(y_true[i])

        for cluster in range(0, self.n_clusters):
            self.temperature[cluster] = self.fitHelper(np.array(_model_logits[cluster]), np.array(_y_true[cluster]), cluster)

    def calibrate(self, probs):
        clipped_model_probs = np.clip(probs, 1e-50, 1)
        model_logits = np.log(clipped_model_probs)
        labels = self.kmeans.predict(model_logits)

        for i in range(model_logits.shape[0]):
            probs[i] = probs[i] ** (1. / self.temperature[labels[i]])

        # calibrated_probs = probs ** (1. / self.temperature)  # Temper
        probs /= np.sum(probs, axis=1, keepdims=True)  # Normalize
        return probs

In [18]:
class OracleCombiner:
    """ Implements the P+L combination method, fit using maximum likelihood
    """
    def __init__(self, calibration_method='temperature scaling', **kwargs):
        self.calibrator = None
        self.confusion_matrix = None  # conf[i, j] is assumed to be P(h = i | Y = j)

        self.n_train_u = None  # Amount of unlabeled training data
        self.n_train_l = None  # Amount of labeled training data
        self.n_cls = None  # Number of classes

        self.eps = 1e-50

        self.use_cv = False
        self.calibration_method = calibration_method
        self.calibrator = TSCalibrator()

    def calibrate(self, model_probs):
        return self.calibrator.calibrate(model_probs)

    def fit(self, model_probs, y_h, y_true):
        self.n_cls = model_probs.shape[1]

        # Estimate human confusion matrix
        # Entry [i, j]  is #(Y = i and h = j)
        conf_h = 1. * confusion_matrix(y_true, y_h, labels=np.arange(self.n_cls))
        # Swap so entry [i, j] is #(h = i and Y = j)
        conf_h = conf_h.T
        conf_h = np.clip(conf_h, self.eps, None)
        normalizer = np.sum(conf_h, axis=0, keepdims=True)
        # Normalize columns so entry [i, j] is P(h = i | Y = j)
        conf_h /= normalizer
        self.confusion_matrix = conf_h

        # Calibrate model probabilities
        if self.use_cv:
            self.fit_calibrator_cv(model_probs, y_true)
        else:
            self.fit_calibrator(model_probs, y_true)

    def fit_bayesian(self, model_probs, y_h, y_true, alpha=0.1, beta=0.1):
        """ This is the "plus one" parameterization, i.e. alpha,beta just need to be > 0
        Really corresponds to a Dirichlet(alpha+1, beta+1, beta+1, . . . ,beta+1) distribution
        """
        self.n_cls = model_probs.shape[1]

        prior_matr = np.eye(self.n_cls) * alpha + (np.ones(self.n_cls) - np.eye(self.n_cls)) * beta

        conf_h = 1. * confusion_matrix(y_true, y_h, labels=np.arange(self.n_cls))
        conf_h += prior_matr
        # Swap so entry [i, j] is #(h = i and Y = j)
        conf_h = conf_h.T
        #conf_h = np.clip(conf_h, self.eps, None)
        normalizer = np.sum(conf_h, axis=0, keepdims=True)
        # Normalize columns so entry [i, j] is P(h = i | Y = j)
        conf_h = conf_h / normalizer
        self.confusion_matrix = conf_h

        # Calibrate model probabilities
        if self.use_cv:
            self.fit_calibrator_cv(model_probs, y_true)
        else:
            self.fit_calibrator(model_probs, y_true)

    def fit_calibrator(self, model_probs, y_true):
        clipped_model_probs = np.clip(model_probs, self.eps, 1)
        model_logits = np.log(clipped_model_probs)
        self.calibrator.fit(model_logits, y_true)

    def fit_calibrator_cv(self, model_probs, y_true):
        # Fits calibration maps that require hyperparameters, using cross-validation
        if self.calibration_method == 'dirichlet':
            reg_lambda_vals = [10., 1., 0., 5e-1, 1e-1, 1e-2, 1e-3]
            skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
            gscv = GridSearchCV(self.calibrator, param_grid={'reg_lambda': reg_lambda_vals,
                                                             'reg_mu': [None]},
                                cv=skf, scoring='neg_log_loss', refit=True)
            gscv.fit(model_probs, y_true)
            self.calibrator = gscv.best_estimator_
        else:
            raise NotImplementedError

    def combine_proba(self, model_probs, y_h, y_true_te, miss_cost = 9, human_cost = 1):
        """ Combines model probabilities with hard labels via the calibrate-confuse equation given the confusion matrix.

        Args:
            p_m: Array of model probabilities ; shape (n_samples, n_classes)
            y_h: List of hard labels ; shape (n_samples,)

        Returns:
            Normalized posterior probabilities P(Y | m, h). Entry [i, j] is P(Y = j | h_i, m_i)
        """
        assert model_probs.shape[0] == y_h.size, 'Size mismatch between model probs and human labels'
        assert model_probs.shape[1] == self.n_cls, 'Size mismatch between model probs and number of classes'

        n_samples = model_probs.shape[0]
        calibrated_model_probs = self.calibrate(model_probs)

        model_output = np.argmax(calibrated_model_probs, axis=1)
        
        # Accuracy Parameters
        human_correct = 0
        human_refered = 0
        model_correct = 0
        model_refered = 0
        
        y_comb = np.empty((n_samples, self.n_cls))
        for i in range(n_samples):

            P_X = miss_cost * (1 - calibrated_model_probs[i][model_output[i]])
            if(human_cost <= P_X):
                # y_comb[i] = calibrated_model_probs[i] * self.confusion_matrix[y_h[i]]
                y_comb[i] = calibrated_model_probs[i] * self.confusion_matrix[y_h[i]]
                human_refered += 1
                if(np.argmax(y_comb[i]) == y_true_te[i]):
                    human_correct += 1
            else:
                y_comb[i] = calibrated_model_probs[i]
                model_refered += 1
                if(np.argmax(y_comb[i]) == y_true_te[i]):
                    model_correct += 1

            if np.allclose(y_comb[i], 0):  # Handle zero rows
                y_comb[i] = np.ones(self.n_cls) * (1./self.n_cls)

        result = {
            'Missclassification Cost' : miss_cost,
            'Human cost': human_cost,
            'Combined accuracy': (human_correct + model_correct) / n_samples,
            'Human correct': human_correct,
            'Human refered': human_refered,
            'Human accuracy': human_correct / human_refered,
            'Model correct': model_correct,
            'Model refered': model_refered,
            'Model accuracy': model_correct / model_refered,
            'Model cost': miss_cost * (model_refered - model_correct) + human_cost * human_refered
        }

        # Don't forget to normalize :)
        assert np.all(np.isfinite(np.sum(y_comb, axis=1)))
        assert np.all(np.sum(y_comb, axis=1) > 0)
        y_comb /= np.sum(y_comb, axis=1, keepdims=True)
        return y_comb, result
        # return calibrated_model_probs, self.confusion_matrix, y_comb

    def combine(self, model_probs, y_h, y_true_te, miss_cost = 9, human_cost = 1):
        """ Combines model probs and y_h to return hard labels
        """
        y_comb_soft, result = self.combine_proba(model_probs, y_h, y_true_te, miss_cost, human_cost)
        return np.argmax(y_comb_soft, axis=1), result

        # calibrated_model_probs, confusion_matrix, y_comb_soft = self.combine_proba(model_probs, y_h)
        # return calibrated_model_probs, confusion_matrix, y_comb_soft
        

In [19]:
def get_acc(y_pred, y_true):
    """ Computes the accuracy of predictions.
    If y_pred is 2D, it is assumed that it is a matrix of scores (e.g. probabilities) of shape (n_samples, n_classes)
    """
    if y_pred.ndim == 1:
        return np.mean(y_pred == y_true)
    elif y_pred.ndim == 2:
        return np.mean(np.argmax(y_pred, axis=1), y_true)

In [20]:
def simulate_single_human(human_counts, seed=0):
    rng = np.random.default_rng(seed)

    human_labels_per_input = np.sum(human_counts, axis=1)
    min_human_labels = int(min(human_labels_per_input))
    n_rows = human_counts.shape[0]
    n_classes = human_counts.shape[1]

    human_labels = np.empty(shape=(n_rows, min_human_labels))
    for row in range(n_rows):
        temp = []
        for i in range(n_classes):
            temp += [i] * int(human_counts[row, i])
        rng.shuffle(temp)
        human_labels[row, :] = temp[:min_human_labels]

    return human_labels[:, 0].astype(int)

In [21]:
def defer_module(model_probs_te, y_h_te, y_true_te):    
    # Human or model
    combined_model_acc = []
    for human_cost in tqdm([1, 5, 10, 20, 30, 40, 50, 60, 70, 80]):
            miss_cost = 100
            human_correct = 0
            human_refered = 0
            model_correct = 0
            model_refered = 0
            miss_cost_model = 0
            miss_cost_combined = 0

            Y_pred = np.argmax(model_probs_te, axis=1)
            y_comb = []
            for i in range(Y_pred.shape[0]):
                P_X = miss_cost * (1 - model_probs_te[i][Y_pred[i]])
                if(human_cost <= P_X):
                    miss_cost_combined += human_cost + miss_cost   # human cost is always included no matter what output it gives
                    if(y_h_te[i] == y_true_te[i]):
                        human_correct += 1
                        miss_cost_combined -= miss_cost
                    human_refered += 1
                    y_comb.append(y_h_te[i])
                else:
                    miss_cost_combined += miss_cost
                    if(Y_pred[i] == y_true_te[i]):
                        miss_cost_combined -= miss_cost
                        model_correct += 1
                    model_refered += 1
                    y_comb.append(Y_pred[i])

                if(Y_pred[i] != y_true_te[i]):
                    miss_cost_model += miss_cost

            acc = np.mean(y_comb == y_true_te)


            combined_model_acc.append([human_cost/miss_cost ,acc, human_refered])
    return combined_model_acc

In [22]:
def calibrate_probability(model_probs_tr, y_true_tr, model_probs_te):
    eps = 1e-50
    k = 70
    calibrator = TSCalibrator(n_clusters=1)
    clipped_model_probs = np.clip(model_probs_tr, eps, 1)
    model_logits = np.log(clipped_model_probs)
    model_logits = np.array(model_logits)
    T = calibrator.fitHelper(model_logits, y_true_tr, 0)

    calibrated_probs = model_probs_te ** (1. / T)  # Temper
    calibrated_probs /= np.sum(calibrated_probs, axis=1, keepdims=True)  # Normalize
    return calibrated_probs

In [23]:
def clustering(model_probs, y_true, y_h, model_probs_te):
    eps = 1e-50
    k = 70
    calibrator = TSCalibrator(n_clusters=k)
    clipped_model_probs = np.clip(model_probs, eps, 1)
    model_logits = np.log(clipped_model_probs)

    kmeans = KMeans(n_clusters=k, random_state=0).fit(model_logits)
  
    _model_logits = []
    _model_probs = []
    _y_h = []
    _y_true = []
    _T = []
    _conf_h = []
    # initialise
    for _ in range(k):
        _model_logits.append([])
        _model_probs.append([])
        _y_h.append([])
        _y_true.append([])
        calibrator.temperature.append(1.0)
        _T.append(1)

    # group based on clusters    
    for i in range(kmeans.labels_.shape[0]):
        cluster_center = kmeans.labels_[i]
        _model_logits[cluster_center].append(model_logits[i])
        _y_true[cluster_center].append(y_true[i])
        _model_probs[cluster_center].append(model_probs[i])
        _y_h[cluster_center].append(y_h[i])


    # Calculater T corresponding to each cluster and confusion matrix for each cluster
    for cluster in range(0, k):
        _T[cluster] = calibrator.fitHelper(np.array(_model_logits[cluster]), np.array(_y_true[cluster]), cluster)
        
        conf_h = 1. * confusion_matrix(np.array(_y_true[cluster]), np.array(_y_h[cluster]), labels=np.arange(10))
        # Swap so entry [i, j] is #(h = i and Y = j)
        conf_h = conf_h.T
        conf_h = np.clip(conf_h, 1e-50, None)
        normalizer = np.sum(conf_h, axis=0, keepdims=True)
        # Normalize columns so entry [i, j] is P(h = i | Y = j)
        conf_h /= normalizer
        _conf_h.append(conf_h)

    clipped_model_probs = np.clip(model_probs_te, eps, 1)
    model_logits = np.log(clipped_model_probs)
    labels = kmeans.predict(model_logits)

    for i in range(model_probs_te.shape[0]):
        model_probs_te[i] = model_probs_te[i] ** (1. / _T[labels[i]])

    model_probs_te /= np.sum(model_probs_te, axis=1, keepdims=True)  # Normalize

    return model_probs_te, _conf_h, labels

In [24]:
def human_in_loop(y_true, y_h, y_true_te, model_probs_te, y_h_te):
    conf_h = 1. * confusion_matrix(y_true, y_h, labels=np.arange(10))
    # Swap so entry [i, j] is #(h = i and Y = j)
    conf_h = conf_h.T
    conf_h = np.clip(conf_h, 1e-50, None)
    normalizer = np.sum(conf_h, axis=0, keepdims=True)
    # Normalize columns so entry [i, j] is P(h = i | Y = j)
    conf_h /= normalizer

    size = y_true_te.shape[0]
    y_comb = np.empty((size, 10))
    Y_pred = np.argmax(model_probs_te, axis=1)

    # Human or model
    combined_model_acc = []
    for human_cost in tqdm([1, 5, 10, 20, 30, 40, 50, 60, 70, 80]):
            miss_cost = 100
            human_correct = 0
            human_refered = 0
            model_correct = 0
            model_refered = 0
            miss_cost_model = 0
            miss_cost_combined = 0

            for i in range(size):
                P_X = miss_cost * (1 - model_probs_te[i][Y_pred[i]])
                if(human_cost <= P_X):
                    y_comb[i] = model_probs_te[i] * conf_h[y_h_te[i]]
                    human_refered += 1
                else: 
                    y_comb[i] = model_probs_te[i]
                    model_refered += 1

                if np.allclose(y_comb[i], 0):  # Handle zero rows
                    y_comb[i] = np.ones(10) * (1./10)

            acc = np.mean(np.argmax(y_comb, axis=1) == y_true_te)

            combined_model_acc.append([human_cost/miss_cost ,acc, human_refered])

    return combined_model_acc

In [25]:
def group_wise_confusion_matrix(size, model_probs_te, _conf_h, labels, y_h_te, y_true_te):
    # model_probs_te, size, 
    y_comb = np.empty((size, 10))
    Y_pred = np.argmax(model_probs_te, axis=1)

    # Human or model
    combined_model_acc = []
    for human_cost in tqdm([1, 5, 10, 20, 30, 40, 50, 60, 70, 80]):
            miss_cost = 100
            human_correct = 0
            human_refered = 0
            model_correct = 0
            model_refered = 0
            miss_cost_model = 0
            miss_cost_combined = 0

            for i in range(size):
                P_X = miss_cost * (1 - model_probs_te[i][Y_pred[i]])
                if(human_cost <= P_X):
                    y_comb[i] = model_probs_te[i] * _conf_h[labels[i]][y_h_te[i]]
                    human_refered += 1
                else:
                    y_comb[i] = model_probs_te[i]
                    model_refered += 1

                if np.allclose(y_comb[i], 0):  # Handle zero rows
                    y_comb[i] = np.ones(10) * (1./10)

            acc = np.mean(np.argmax(y_comb, axis=1) == y_true_te)
            combined_model_acc.append([human_cost/miss_cost, acc, human_refered])
    return combined_model_acc

In [26]:
def print_results(combined_model_acc):
    print('C2D\tCombined Model Accuracy\tNumber of Defers')
    for c2d, acc, defer in combined_model_acc:
        print('{}\t{}%\t{}'.format(c2d, acc*100, defer))

In [27]:
def run_experiment_cifar10(out_fpath=None, experiment_args=None, method='m1', seed=0):
    model_names = [
                   'r_low_acc', 
                   'resnet-110', 
                    'preresnet-110', 
                   'densenet-bc-L190-k40'
                   ]

    for model_name in tqdm(model_names, desc='Models', leave=True):
        # Load data
        human_counts, model_probs, y_true = load_CIFAR10H(model_name) # data_utils.py
        y_h = simulate_single_human(human_counts, seed=seed)

        y_h_tr, y_h_te, model_probs_tr, model_probs_te, y_true_tr, y_true_te = train_test_split(y_h, model_probs, y_true, test_size=0.5, random_state=1 * seed)
        print(model_name)
        print('Training set size: {}'.format(y_h_tr.shape[0]))
        print('Expert Accuracy: {}'.format(np.mean(y_h_te == y_true_te)))    
        print('Machine learning model accuracy: {}'.format(np.mean(np.argmax(model_probs_te, axis=1) == y_true_te)))
        
        if method == 'm1':
            # Basic Defer Model
            combined_model_acc1 = defer_module(model_probs_te, y_h_te, y_true_te)
            print_results(combined_model_acc1)
        elif method == 'm2':
            calibrated_probs = calibrate_probability(model_probs_tr, y_true_tr, model_probs_te)
            combined_model_acc = defer_module(calibrated_probs, y_h_te, y_true_te)
            print_results(combined_model_acc)
        elif method == 'm3':
            model_probs_te, _, _ = clustering(model_probs, y_true, y_h, model_probs_te)
            combined_model_acc3 = defer_module(model_probs_te, y_h_te, y_true_te)
            print_results(combined_model_acc3)
        elif method == 'm4':
            model_probs_te, _, _ = clustering(model_probs, y_true, y_h, model_probs_te)
            combined_model_acc4 = human_in_loop(y_true, y_h, y_true_te, model_probs_te, y_h_te)
            print_results(combined_model_acc4)
        elif method == 'm5':
            size = y_true_te.shape[0]
            model_probs_te, _conf_h, labels = clustering(model_probs, y_true, y_h, model_probs_te)
            combined_model_acc5 = group_wise_confusion_matrix(size, model_probs_te, _conf_h, labels, y_h_te, y_true_te)
            print_results(combined_model_acc5)
        print('\n\n')


### Setup

In [28]:
seed = 9658
torch.manual_seed(seed)
np.random.seed(seed)
calibration_methods = ['temperature scaling']
args = {'n_runs': 1, 'test_size': 0.3, 'calibration_methods': calibration_methods, 'seed': seed}

In [29]:
out_fpath = './output/'
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, method='m1', seed=seed)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

r_low_acc
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.8848


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	95.88%	1678
0.05	95.14%	1140
0.1	94.42%	877
0.2	93.34%	628
0.3	91.92%	442
0.4	90.72%	263
0.5	89.44%	97
0.6	88.58%	17
0.7	88.52%	2
0.8	88.48%	0



resnet-110
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.9402


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	96.98%	946
0.05	96.56%	592
0.1	96.22%	458
0.2	95.64%	291
0.3	95.26%	208
0.4	94.72%	131
0.5	94.28%	36
0.6	94.1%	9
0.7	94.04%	1
0.8	94.02000000000001%	0



preresnet-110
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.9508


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	97.44%	622
0.05	97.02%	402
0.1	96.88%	301
0.2	96.44%	195
0.3	96.1%	135
0.4	95.66%	69
0.5	95.24000000000001%	16
0.6	95.1%	1
0.7	95.08%	0
0.8	95.08%	0



densenet-bc-L190-k40
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.966


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	97.96000000000001%	348
0.05	97.74000000000001%	209
0.1	97.48%	161
0.2	97.16%	104
0.3	97.0%	69
0.4	96.92%	44
0.5	96.66%	12
0.6	96.61999999999999%	2
0.7	96.6%	0
0.8	96.6%	0





### METHOD-2: Single calibration papameter

In [30]:
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, method='m2', seed=seed)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

r_low_acc
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.8848


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	95.78%	2920
0.05	95.92%	2008
0.1	95.89999999999999%	1595
0.2	95.14%	1115
0.3	94.06%	792
0.4	92.9%	556
0.5	91.08000000000001%	299
0.6	89.48%	102
0.7	88.58%	7
0.8	88.48%	0



resnet-110
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.9402


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	96.67999999999999%	1941
0.05	96.89999999999999%	1226
0.1	96.92%	909
0.2	96.54%	585
0.3	96.24000000000001%	411
0.4	95.42%	264
0.5	94.82000000000001%	133
0.6	94.34%	41
0.7	94.06%	5
0.8	94.02000000000001%	0



preresnet-110
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.9508


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	96.89999999999999%	1704
0.05	97.54%	969
0.1	97.39999999999999%	696
0.2	97.22%	463
0.3	96.96000000000001%	322
0.4	96.5%	217
0.5	95.78%	81
0.6	95.26%	22
0.7	95.12%	3
0.8	95.08%	0



densenet-bc-L190-k40
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.966


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	96.94%	1941
0.05	98.04%	503
0.1	98.04%	337
0.2	97.68%	209
0.3	97.38%	144
0.4	97.14%	90
0.5	96.84%	39
0.6	96.67999999999999%	11
0.7	96.61999999999999%	2
0.8	96.6%	0





### METHOD:3

In [31]:
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, method='m3', seed=seed)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

r_low_acc
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.8848


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	95.78%	3002
0.05	96.24000000000001%	1987
0.1	95.74000000000001%	1563
0.2	94.96%	1089
0.3	94.1%	791
0.4	92.80000000000001%	548
0.5	90.98%	289
0.6	89.5%	103
0.7	88.58%	8
0.8	88.48%	0



resnet-110
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.9402


...

  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	96.94%	1820
0.05	97.5%	941
0.1	97.58%	689
0.2	97.24000000000001%	472
0.3	96.92%	318
0.4	96.52%	197
0.5	95.67999999999999%	80
0.6	95.26%	19
0.7	95.14%	3
0.8	95.08%	0



densenet-bc-L190-k40
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.966


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	97.18%	1426
0.05	98.06%	566
0.1	98.08%	386
0.2	97.8%	232
0.3	97.6%	166
0.4	97.18%	100
0.5	96.76%	43
0.6	96.67999999999999%	12
0.7	96.6%	1
0.8	96.6%	0





### Method 4

In [32]:
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, method='m4', seed=seed)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

r_low_acc
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.8848


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	96.52%	3002
0.05	96.56%	1987
0.1	96.0%	1563
0.2	95.06%	1089
0.3	94.16%	791
0.4	92.86%	548
0.5	90.98%	289
0.6	89.52%	103
0.7	88.58%	8
0.8	88.48%	0



resnet-110
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.9402


...

  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	97.88%	1820
0.05	97.89999999999999%	941
0.1	97.84%	689
0.2	97.36%	472
0.3	96.96000000000001%	318
0.4	96.5%	197
0.5	95.67999999999999%	80
0.6	95.24000000000001%	19
0.7	95.14%	3
0.8	95.08%	0



densenet-bc-L190-k40
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.966


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	98.16%	1426
0.05	98.16%	566
0.1	98.08%	386
0.2	97.82%	232
0.3	97.61999999999999%	166
0.4	97.18%	100
0.5	96.76%	43
0.6	96.67999999999999%	12
0.7	96.6%	1
0.8	96.6%	0





### Method 5: Group wise confusion matrix implementation


In [33]:
run_experiment_cifar10(out_fpath=out_fpath, experiment_args=args, method='m5', seed=seed)

Models:   0%|          | 0/4 [00:00<?, ?it/s]

r_low_acc
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.8848


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	97.08%	3002
0.05	96.84%	1987
0.1	96.24000000000001%	1563
0.2	95.24000000000001%	1089
0.3	94.34%	791
0.4	93.02%	548
0.5	91.12%	289
0.6	89.53999999999999%	103
0.7	88.58%	8
0.8	88.48%	0



resnet-110
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.9402


...

  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	98.11999999999999%	1820
0.05	98.02%	941
0.1	97.94%	689
0.2	97.44%	472
0.3	97.0%	318
0.4	96.5%	197
0.5	95.7%	80
0.6	95.24000000000001%	19
0.7	95.12%	3
0.8	95.08%	0



densenet-bc-L190-k40
Training set size: 5000
Expert Accuracy: 0.9518
Machine learning model accuracy: 0.966


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

C2D	Combined Model Accuracy	Number of Defers
0.01	98.6%	1426
0.05	98.52%	566
0.1	98.38%	386
0.2	98.1%	232
0.3	97.82%	166
0.4	97.34%	100
0.5	96.84%	43
0.6	96.72%	12
0.7	96.6%	1
0.8	96.6%	0



